In [1]:
import sys, os
sys.path.append(os.path.abspath('/spark-data'))

In [58]:
#Importing Spark Session and important libraries that need to perform things in project
from utils.Data_Ingestion.data_ingestion import load_files 
from utils.Data_Cleaning.data_cleaning import remove_duplicates
from utils.Data_Cleaning.data_cleaning import standardize_date_format, remove_decimal, clean_phone_numbers, handle_negative_values, remove_string_from_column, impute_nulls, drop_nulls
from utils.save_dataframe_to_files import save_file
from pyspark.sql import SparkSession

In [3]:
#Spark Session Builder that builds the Session in which we work upon

spark = SparkSession.builder \
    .appName("ExceltoDataFrame") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:3.4.1_0.19.0") \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.crealytics#spark-excel_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f6898633-c5b8-4da8-88e6-01041afbb5ad;1.0
	confs: [default]
	found com.crealytics#spark-excel_2.12;3.4.1_0.19.0 in central
	found org.apache.poi#poi;5.2.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.commons#commons-collections4;4.4 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found com.zaxxer#SparseBitSet;1.2 in central
	found org.apache.poi#poi-ooxml;5.2.3 in central
	found org.apache.poi#poi-ooxml-lite;5.2.3 in central
	found org.apache.xmlbeans#xmlbeans;5.1.1 in central
	found com.github.virtuald#curvesapi;1.07 in central
	found com.norbitltd#spoiwo_2.12;2.2.1 in central
	found com.github.tototoshi#scala-csv_2.12;1.3.10 in central
	found com.github.pjfanning#excel-streaming-reader;4.0.5 in central
	found com.gi

In [4]:
dfs = load_files(spark)
for df_name, df in dfs.items():
    print(f"DataFrame Name: {df_name, df}")

2024-08-30 15:57:07,671 - INFO - Successfully processed nested JSON field 'orders' in file: /spark-data/Data_Files/data.json
2024-08-30 15:57:07,752 - INFO - Successfully processed nested JSON field 'reviews' in file: /spark-data/Data_Files/data.json
2024-08-30 15:57:10,629 - INFO - Successfully loaded sheet 'data' from Excel file: /spark-data/Data_Files/data.xlsx
2024-08-30 15:57:11,657 - INFO - Successfully loaded sheet 'product' from Excel file: /spark-data/Data_Files/data.xlsx
2024-08-30 15:57:12,375 - INFO - Successfully loaded sheet 'customer' from Excel file: /spark-data/Data_Files/data.xlsx


DataFrame Name: ('df_data_orders', DataFrame[order_id: bigint, order_item_id: bigint, price: bigint, product_id: bigint, quantity: bigint])
DataFrame Name: ('df_data_reviews', DataFrame[customer_id: bigint, product_id: bigint, rating: bigint, review_date: string, review_id: bigint, review_text: string])
DataFrame Name: ('df_data_data', DataFrame[order_id: double, customer_id: double, order_date: timestamp, total_amount: double, shipping_address: string, billing_address: string])
DataFrame Name: ('df_data_product', DataFrame[product_id: double, product_name: string, category: string, price: double, description: string, inventory: double])
DataFrame Name: ('df_data_customer', DataFrame[customer_id: double, first_name: string, last_name: string, email: string, phone_number: string, city: string, state: string, zip_code: double, birth_date: timestamp])


In [5]:
dfs['df_data_orders'].show()

+--------+-------------+-----+----------+--------+
|order_id|order_item_id|price|product_id|quantity|
+--------+-------------+-----+----------+--------+
|     868|            1|   70|      1571|       7|
|    3151|            2|    8|      3809|      -3|
|    1582|            3|   32|      4896|       8|
|    2578|            4|   64|      2712|       4|
|    2792|            5|   53|      1111|       4|
|    1257|            6|   69|      2874|       1|
|    2389|            7|   40|      2670|       9|
|    2013|            8|   39|      3779|       6|
|    4272|            9|   15|      3109|       5|
|    1260|           10|   21|      2110|       8|
|    3617|           11|   27|      3350|       1|
|    1479|           12|   96|      2917|       9|
|    1405|           13|   61|      1480|       5|
|    4526|           14|   69|      4686|       5|
|    2513|           15|    8|      3146|       8|
|    2346|           16|   92|      4182|       5|
|    3988|           17|   88| 

In [6]:
dfs['df_data_reviews'].show()

+-----------+----------+------+-----------+---------+--------------------+
|customer_id|product_id|rating|review_date|review_id|         review_text|
+-----------+----------+------+-----------+---------+--------------------+
|       2522|      4663|     3| 2022-12-20|        1|Home create anyon...|
|       2181|      2645|     3| 2024-07-05|        2|Skill herself opt...|
|       3935|      1699|     3| 2024-05-06|        3|Public everyone s...|
|       3453|      1360|     1| 2023-11-05|        4|Level land majori...|
|       2999|      4883|     3| 2024-03-03|        5|Treat fine now di...|
|        863|      3544|     2| 2023-06-02|        6|Land despite buy ...|
|       2348|      1853|     1| 2023-11-11|        7|Ground again goal...|
|       2388|      1652|     3| 2022-11-29|        8|Community materia...|
|       4486|       632|     4| 2023-03-31|        9|Goal after color ...|
|       1379|      2399|     5| 2024-02-11|       10|Beyond coach meet...|
|        309|      3241| 

In [7]:
dfs['df_data_data'].show()

+--------+-----------+-------------------+------------+--------------------+--------------------+
|order_id|customer_id|         order_date|total_amount|    shipping_address|     billing_address|
+--------+-----------+-------------------+------------+--------------------+--------------------+
|     1.0|      462.0|2019-08-26 00:00:00|      4007.0|60585 Moore Overp...|775 Roberson Knol...|
|     2.0|     2820.0|2021-04-22 00:00:00|      7240.0|3278 Price Isle\n...|2055 Curry Street...|
|     3.0|     1934.0|2024-02-21 00:00:00|      8318.0|Unit 8425 Box 079...|USNV Strong\nFPO ...|
|     4.0|     2694.0|2021-01-14 00:00:00|      4992.0|5566 Brittany Sta...|524 Allen Shoal S...|
|     5.0|      134.0|2020-03-14 00:00:00|      5385.0|4289 Michael Esta...|38637 Cheryl Junc...|
|     6.0|     4651.0|2020-12-30 00:00:00|      3512.0|83184 Shelly Land...|80329 Dawn Road S...|
|     7.0|     1696.0|2023-05-30 00:00:00|      4592.0|7127 Arroyo Land\...|787 Smith Islands...|
|     8.0|     1969.

In [8]:
dfs['df_data_product'].show()

+----------+-------------+---------+-----+--------------------+---------+
|product_id| product_name| category|price|         description|inventory|
+----------+-------------+---------+-----+--------------------+---------+
|       1.0|          bag|       no|448.0|Summer now health...|    655.0|
|       2.0|        happy|   answer|844.0|Employee discuss ...|    230.0|
|       3.0|         same|      his|681.0|Million mother pu...|    831.0|
|       4.0|      prevent|   around| 34.0|Because under mea...|    804.0|
|       5.0|        quite|   summer|663.0|Approach Mr task ...|    351.0|
|       6.0|        which|  purpose|386.0|Leave charge mode...|     10.0|
|       7.0|       beyond|   beyond|879.0|Shoulder power in...|    282.0|
|       8.0|     although|     home|650.0|Exactly continue ...|    818.0|
|       9.0|environmental| although|  7.0|Really admit kind...|    128.0|
|      10.0|       assume|professor|945.0|Bar truth here gr...|    168.0|
|      11.0|      usually|   figure|43

### Cleaning df_customer

In [9]:
df_customer = dfs['df_data_customer']

In [10]:
df_customer.show()

+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+-------------------+
|customer_id| first_name|last_name|               email|        phone_number|             city|        state|zip_code|         birth_date|
+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+-------------------+
|        1.0|    Crystal|  Edwards|mcmahonemily@exam...|        511.782.4381|    Fergusonville|         Iowa| 91344.0|1995-02-28 00:00:00|
|        2.0|   Jennifer|    Jones|douglasedwards@ex...|          9187202495|     Jenniferport|   New Jersey| 59371.0|1951-01-27 00:00:00|
|        3.0|  Catherine|    Crane|douglaskevin@exam...|  (381)601-2272x7972|  Jenniferchester|West Virginia| 49812.0|1966-10-07 00:00:00|
|        4.0|     Lauren|    Irwin|aguirresteven@exa...|    001-707-942-4829|      New Melissa|     Michigan| 17279.0|1936-07-28 00:00:00|
|        5.0|Christopher|  

In [11]:
#This is for when you want to apply this remove_duplicate on specific column of dataframe but it is optional
# df_customer_demo = remove_duplicates(df_customer, columns=["customer_id"]).show()

#For remove Duplicates on df_customer dataframe
df_customer_remove_duplicates = remove_duplicates(df_customer)

2024-08-30 15:57:19,086 - INFO - Successfully removed duplicate rows based on all columns.


In [12]:
df_customer_correct_birthdate = standardize_date_format(df_customer_remove_duplicates, 'birth_date')


2024-08-30 15:57:19,334 - INFO - Successfully standardized date format in column 'birth_date' to 'yyyy-MM-dd'.


In [13]:
df_customer_correct_birthdate.show()

+-----------+----------+----------+--------------------+--------------------+------------------+--------------+--------+----------+
|customer_id|first_name| last_name|               email|        phone_number|              city|         state|zip_code|birth_date|
+-----------+----------+----------+--------------------+--------------------+------------------+--------------+--------+----------+
|       25.0|     Heidi|Montgomery|rachel77@example.net|001-400-843-7979x...|       East Robert|North Carolina| 88566.0|2009-01-23|
|      177.0|   Natalie|     Huynh|hector04@example.com| +1-882-707-3414x569|    West Markmouth|North Carolina| 47069.0|1936-09-17|
|      295.0|       Amy|     Stone|deniseanderson@ex...|        909-267-3565|Port Christianstad|   Connecticut| 99042.0|1975-10-12|
|      336.0|     Terri|    Warren| wmorrow@example.net|   473.898.3514x8669|   Lake Andrewport|  South Dakota| 79840.0|2020-01-05|
|      130.0|   Suzanne|   Rowland|  nbrown@example.org|+1-445-450-7790x1...

In [14]:
df_customer_remove_decimal = remove_decimal(df_customer_correct_birthdate, columns=["zip_code", "customer_id"])


2024-08-30 15:57:22,036 - INFO - Successfully removed decimal values from column 'zip_code'.


2024-08-30 15:57:22,077 - INFO - Successfully removed decimal values from column 'customer_id'.


In [15]:
df_customer_remove_decimal.show()

+-----------+----------+----------+--------------------+--------------------+------------------+--------------+--------+----------+
|customer_id|first_name| last_name|               email|        phone_number|              city|         state|zip_code|birth_date|
+-----------+----------+----------+--------------------+--------------------+------------------+--------------+--------+----------+
|         25|     Heidi|Montgomery|rachel77@example.net|001-400-843-7979x...|       East Robert|North Carolina|   88566|2009-01-23|
|        177|   Natalie|     Huynh|hector04@example.com| +1-882-707-3414x569|    West Markmouth|North Carolina|   47069|1936-09-17|
|        295|       Amy|     Stone|deniseanderson@ex...|        909-267-3565|Port Christianstad|   Connecticut|   99042|1975-10-12|
|        336|     Terri|    Warren| wmorrow@example.net|   473.898.3514x8669|   Lake Andrewport|  South Dakota|   79840|2020-01-05|
|        130|   Suzanne|   Rowland|  nbrown@example.org|+1-445-450-7790x1...

In [16]:
df_customer_clean_phone_numbers = clean_phone_numbers(df_customer_remove_decimal, 'phone_number')

In [17]:
df_customer_clean_phone_numbers.show()

+-----------+----------+----------+--------------------+--------------------+------------------+--------------+--------+----------+
|customer_id|first_name| last_name|               email|        phone_number|              city|         state|zip_code|birth_date|
+-----------+----------+----------+--------------------+--------------------+------------------+--------------+--------+----------+
|         25|     Heidi|Montgomery|rachel77@example.net|Invalid phone number|       East Robert|North Carolina|   88566|2009-01-23|
|        177|   Natalie|     Huynh|hector04@example.com|+1 (882) 707-3414...|    West Markmouth|North Carolina|   47069|1936-09-17|
|        295|       Amy|     Stone|deniseanderson@ex...|   +1 (909) 267-3565|Port Christianstad|   Connecticut|   99042|1975-10-12|
|        336|     Terri|    Warren| wmorrow@example.net|+1 (473) 898-3514...|   Lake Andrewport|  South Dakota|   79840|2020-01-05|
|        130|   Suzanne|   Rowland|  nbrown@example.org|+1 (445) 450-7790...

In [18]:
df_customer_cleaned = df_customer_clean_phone_numbers
df_customer_cleaned.show()

+-----------+----------+----------+--------------------+--------------------+------------------+--------------+--------+----------+
|customer_id|first_name| last_name|               email|        phone_number|              city|         state|zip_code|birth_date|
+-----------+----------+----------+--------------------+--------------------+------------------+--------------+--------+----------+
|         25|     Heidi|Montgomery|rachel77@example.net|Invalid phone number|       East Robert|North Carolina|   88566|2009-01-23|
|        177|   Natalie|     Huynh|hector04@example.com|+1 (882) 707-3414...|    West Markmouth|North Carolina|   47069|1936-09-17|
|        295|       Amy|     Stone|deniseanderson@ex...|   +1 (909) 267-3565|Port Christianstad|   Connecticut|   99042|1975-10-12|
|        336|     Terri|    Warren| wmorrow@example.net|+1 (473) 898-3514...|   Lake Andrewport|  South Dakota|   79840|2020-01-05|
|        130|   Suzanne|   Rowland|  nbrown@example.org|+1 (445) 450-7790...

In [36]:
save_file(df_customer_cleaned, "/spark-data/Cleaned_Csv_File", "df_customer_cleaned")

2024-08-30 16:08:28,901 - INFO - Saving DataFrame to temporary directory...
2024-08-30 16:08:31,356 - INFO - DataFrame saved successfully to temporary directory.
2024-08-30 16:08:31,358 - INFO - File moved to final output path: /spark-data/Cleaned_Csv_File/df_customer_cleaned.csv
2024-08-30 16:08:31,360 - INFO - Temporary directory removed: /spark-data/Cleaned_Csv_File/temp_dir


'/spark-data/Cleaned_Csv_File/df_customer_cleaned.csv'

### Cleanining df_data_product table

In [19]:
df_product = dfs['df_data_product']

In [20]:
df_product.show()

+----------+-------------+---------+-----+--------------------+---------+
|product_id| product_name| category|price|         description|inventory|
+----------+-------------+---------+-----+--------------------+---------+
|       1.0|          bag|       no|448.0|Summer now health...|    655.0|
|       2.0|        happy|   answer|844.0|Employee discuss ...|    230.0|
|       3.0|         same|      his|681.0|Million mother pu...|    831.0|
|       4.0|      prevent|   around| 34.0|Because under mea...|    804.0|
|       5.0|        quite|   summer|663.0|Approach Mr task ...|    351.0|
|       6.0|        which|  purpose|386.0|Leave charge mode...|     10.0|
|       7.0|       beyond|   beyond|879.0|Shoulder power in...|    282.0|
|       8.0|     although|     home|650.0|Exactly continue ...|    818.0|
|       9.0|environmental| although|  7.0|Really admit kind...|    128.0|
|      10.0|       assume|professor|945.0|Bar truth here gr...|    168.0|
|      11.0|      usually|   figure|43

In [21]:
df_product_remove_decimals = remove_decimal(df_product, columns=["product_id", "inventory"])

2024-08-30 15:57:29,353 - INFO - Successfully removed decimal values from column 'product_id'.
2024-08-30 15:57:29,378 - INFO - Successfully removed decimal values from column 'inventory'.


In [22]:
df_product_remove_decimals.show()

+----------+-------------+---------+-----+--------------------+---------+
|product_id| product_name| category|price|         description|inventory|
+----------+-------------+---------+-----+--------------------+---------+
|         1|          bag|       no|448.0|Summer now health...|      655|
|         2|        happy|   answer|844.0|Employee discuss ...|      230|
|         3|         same|      his|681.0|Million mother pu...|      831|
|         4|      prevent|   around| 34.0|Because under mea...|      804|
|         5|        quite|   summer|663.0|Approach Mr task ...|      351|
|         6|        which|  purpose|386.0|Leave charge mode...|       10|
|         7|       beyond|   beyond|879.0|Shoulder power in...|      282|
|         8|     although|     home|650.0|Exactly continue ...|      818|
|         9|environmental| although|  7.0|Really admit kind...|      128|
|        10|       assume|professor|945.0|Bar truth here gr...|      168|
|        11|      usually|   figure|43

In [23]:
df_product_handling_negative_val = handle_negative_values(df_product_remove_decimals, columns=["price"], operation="absolute")

2024-08-30 15:57:30,077 - INFO - Successfully handled negative values in column 'price' by converting to absolute values.


In [24]:
df_product_handling_negative_val.show()

+----------+-------------+---------+-----+--------------------+---------+
|product_id| product_name| category|price|         description|inventory|
+----------+-------------+---------+-----+--------------------+---------+
|         1|          bag|       no|448.0|Summer now health...|      655|
|         2|        happy|   answer|844.0|Employee discuss ...|      230|
|         3|         same|      his|681.0|Million mother pu...|      831|
|         4|      prevent|   around| 34.0|Because under mea...|      804|
|         5|        quite|   summer|663.0|Approach Mr task ...|      351|
|         6|        which|  purpose|386.0|Leave charge mode...|       10|
|         7|       beyond|   beyond|879.0|Shoulder power in...|      282|
|         8|     although|     home|650.0|Exactly continue ...|      818|
|         9|environmental| although|  7.0|Really admit kind...|      128|
|        10|       assume|professor|945.0|Bar truth here gr...|      168|
|        11|      usually|   figure|43

In [25]:
df_product_handling_negative_val = handle_negative_values(df_product_handling_negative_val, columns=["inventory"], operation='zero')

2024-08-30 15:57:30,942 - INFO - Successfully handled negative values in column 'inventory' by setting them to zero.


In [31]:
df_product_desciption_filter = remove_string_from_column(df_product_handling_negative_val,"description", "\n")

2024-08-30 16:01:59,821 - INFO - Successfully removed '
' from column 'description'.


In [32]:
df_cleaned_product = df_product_desciption_filter

In [33]:
df_cleaned_product.show()

+----------+-------------+---------+-----+--------------------+---------+
|product_id| product_name| category|price|         description|inventory|
+----------+-------------+---------+-----+--------------------+---------+
|         1|          bag|       no|448.0|Summer now health...|      655|
|         2|        happy|   answer|844.0|Employee discuss ...|      230|
|         3|         same|      his|681.0|Million mother pu...|      831|
|         4|      prevent|   around| 34.0|Because under mea...|      804|
|         5|        quite|   summer|663.0|Approach Mr task ...|      351|
|         6|        which|  purpose|386.0|Leave charge mode...|       10|
|         7|       beyond|   beyond|879.0|Shoulder power in...|      282|
|         8|     although|     home|650.0|Exactly continue ...|      818|
|         9|environmental| although|  7.0|Really admit kind...|      128|
|        10|       assume|professor|945.0|Bar truth here gr...|      168|
|        11|      usually|   figure|43

In [34]:
df_cleaned_product.select("description").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|description                                                                                                                                                                                          |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Summer now health each budget me.Before hard later nice for. Meet because door worry.Too be pick effect he wide. Network executive fight somebody discuss science under.                             |
|Employee discuss society fly watch.Exactly affect seven physical big artist there.Into event still region thus left. Will tree box political popular force. Budget hold such as mind individual pick.|


In [37]:
save_file(df_cleaned_product, "/spark-data/Cleaned_Csv_File", "df_product_cleaned")

2024-08-30 16:10:39,051 - INFO - Saving DataFrame to temporary directory...
24/08/30 16:10:40 WARN TaskSetManager: Stage 32 contains a task of very large size (1070 KiB). The maximum recommended task size is 1000 KiB.
2024-08-30 16:10:40,376 - INFO - DataFrame saved successfully to temporary directory.
2024-08-30 16:10:40,379 - INFO - File moved to final output path: /spark-data/Cleaned_Csv_File/df_product_cleaned.csv
2024-08-30 16:10:40,381 - INFO - Temporary directory removed: /spark-data/Cleaned_Csv_File/temp_dir


'/spark-data/Cleaned_Csv_File/df_product_cleaned.csv'

### df_data_data table cleaning


In [38]:
df_data = dfs['df_data_data']

In [39]:
df_data.show()

+--------+-----------+-------------------+------------+--------------------+--------------------+
|order_id|customer_id|         order_date|total_amount|    shipping_address|     billing_address|
+--------+-----------+-------------------+------------+--------------------+--------------------+
|     1.0|      462.0|2019-08-26 00:00:00|      4007.0|60585 Moore Overp...|775 Roberson Knol...|
|     2.0|     2820.0|2021-04-22 00:00:00|      7240.0|3278 Price Isle\n...|2055 Curry Street...|
|     3.0|     1934.0|2024-02-21 00:00:00|      8318.0|Unit 8425 Box 079...|USNV Strong\nFPO ...|
|     4.0|     2694.0|2021-01-14 00:00:00|      4992.0|5566 Brittany Sta...|524 Allen Shoal S...|
|     5.0|      134.0|2020-03-14 00:00:00|      5385.0|4289 Michael Esta...|38637 Cheryl Junc...|
|     6.0|     4651.0|2020-12-30 00:00:00|      3512.0|83184 Shelly Land...|80329 Dawn Road S...|
|     7.0|     1696.0|2023-05-30 00:00:00|      4592.0|7127 Arroyo Land\...|787 Smith Islands...|
|     8.0|     1969.

In [40]:
df_data_removed_duplicate = remove_duplicates(df_data)

2024-08-30 16:43:23,363 - INFO - Successfully removed duplicate rows based on all columns.


In [42]:
df_data_date_formated = standardize_date_format(df_data_removed_duplicate, "order_date")

2024-08-30 16:45:16,276 - INFO - Successfully standardized date format in column 'order_date' to 'yyyy-MM-dd'.


In [43]:
df_data_remove_decimals = remove_decimal(df_data_date_formated, columns=['order_id', 'customer_id', 'total_amount'])

2024-08-30 16:46:37,618 - INFO - Successfully removed decimal values from column 'order_id'.
2024-08-30 16:46:37,635 - INFO - Successfully removed decimal values from column 'customer_id'.
2024-08-30 16:46:37,654 - INFO - Successfully removed decimal values from column 'total_amount'.


In [44]:
df_data_negative_handel = handle_negative_values(df_data_remove_decimals, columns=["order_id", "customer_id", "total_amount"], operation="absolute")

2024-08-30 16:53:23,947 - INFO - Successfully handled negative values in column 'order_id' by converting to absolute values.
2024-08-30 16:53:23,993 - INFO - Successfully handled negative values in column 'customer_id' by converting to absolute values.
2024-08-30 16:53:24,045 - INFO - Successfully handled negative values in column 'total_amount' by converting to absolute values.


In [49]:
df_handel_address = remove_string_from_column(df_data_negative_handel, "shipping_address", "\n")
df_handel_address = remove_string_from_column(df_data_negative_handel, "billing_address", "\n")

2024-08-30 16:57:32,794 - INFO - Successfully removed '
' from column 'shipping_address'.
2024-08-30 16:57:32,810 - INFO - Successfully removed '
' from column 'billing_address'.


In [50]:
df_data_cleaned = df_handel_address

In [51]:
df_data_cleaned.show()

+--------+-----------+----------+------------+--------------------+--------------------+
|order_id|customer_id|order_date|total_amount|    shipping_address|     billing_address|
+--------+-----------+----------+------------+--------------------+--------------------+
|     131|       4958|2020-05-10|         569|7886 Higgins Moto...|14788 Robert Land...|
|     297|         46|2020-03-11|        1331|PSC 4470, Box 299...|16387 Novak Shoal...|
|     330|       1082|2025-05-03|        4010|758 Sarah Plaza\n...|938 Brian RoadLak...|
|      62|       2934|2024-06-01|        1727|7971 Vicki Key Ap...|465 Kramer Way Ap...|
|     266|       3929|2022-05-30|         680|57853 Chen Way\nL...|055 Butler Extens...|
|      43|        118|2021-09-26|        4884|9822 Anne Trace A...|1511 Hill Keys Ap...|
|      95|       4222|2023-08-27|        3765|Unit 4050 Box 161...|06811 Sean Crossi...|
|     167|       3800|2022-12-09|        6259|16060 Mason Creek...|196 Joshua Meadow...|
|     300|       1337

In [52]:
save_file(df_data_cleaned, "/spark-data/Cleaned_Csv_File","df_data_cleaned")

2024-08-30 17:04:04,507 - INFO - Saving DataFrame to temporary directory...
2024-08-30 17:04:05,945 - INFO - DataFrame saved successfully to temporary directory.
2024-08-30 17:04:05,949 - INFO - File moved to final output path: /spark-data/Cleaned_Csv_File/df_data_cleaned.csv
2024-08-30 17:04:05,953 - INFO - Temporary directory removed: /spark-data/Cleaned_Csv_File/temp_dir


'/spark-data/Cleaned_Csv_File/df_data_cleaned.csv'

### df_data_reviews cleaning

In [53]:
df_review = dfs["df_data_reviews"]

In [54]:
df_review.show()

+-----------+----------+------+-----------+---------+--------------------+
|customer_id|product_id|rating|review_date|review_id|         review_text|
+-----------+----------+------+-----------+---------+--------------------+
|       2522|      4663|     3| 2022-12-20|        1|Home create anyon...|
|       2181|      2645|     3| 2024-07-05|        2|Skill herself opt...|
|       3935|      1699|     3| 2024-05-06|        3|Public everyone s...|
|       3453|      1360|     1| 2023-11-05|        4|Level land majori...|
|       2999|      4883|     3| 2024-03-03|        5|Treat fine now di...|
|        863|      3544|     2| 2023-06-02|        6|Land despite buy ...|
|       2348|      1853|     1| 2023-11-11|        7|Ground again goal...|
|       2388|      1652|     3| 2022-11-29|        8|Community materia...|
|       4486|       632|     4| 2023-03-31|        9|Goal after color ...|
|       1379|      2399|     5| 2024-02-11|       10|Beyond coach meet...|
|        309|      3241| 

In [57]:
df_review_remove_duplicate = remove_duplicates(df_review)

2024-08-30 17:07:47,777 - INFO - Successfully removed duplicate rows based on all columns.


In [59]:
df_review_null_handel = drop_nulls(df_review_remove_duplicate, "any","review_id")

In [62]:
df_review_null_handel.select("review_id").count()

10000

In [64]:
df_review_review_handel = remove_string_from_column(df_review_null_handel, "review_text", "\n")

2024-08-30 17:32:06,216 - INFO - Successfully removed '
' from column 'review_text'.


In [65]:
df_review_review_handel.show()

+-----------+----------+------+-----------+---------+--------------------+
|customer_id|product_id|rating|review_date|review_id|         review_text|
+-----------+----------+------+-----------+---------+--------------------+
|        166|      3108|     1| 2024-02-12|      363|Realize option ap...|
|       1662|      3924|     5| 2024-05-21|      493|Goal rich travel ...|
|       4238|       281|     4| 2023-01-01|      619|Fish future kind ...|
|        520|       498|     2| 2023-07-03|      678|Computer itself t...|
|         58|      1360|     1| 2024-02-01|     1290|Into upon enough ...|
|       2277|      3726|     3| 2023-12-11|     1580|Word peace race m...|
|       3290|      2684|     5| 2024-06-20|     1627|Market process pr...|
|       4476|      2171|     1| 2023-07-07|     1939|South song race p...|
|         63|      4951|     2| 2022-12-29|     2547|Tv character seni...|
|         84|       983|     2| 2024-03-12|     2572|Six the admit bes...|
|        572|       416| 

In [66]:
df_review_cleaned = df_review_review_handel

In [67]:
save_file(df_review_cleaned, "/spark-data/Cleaned_Csv_File", "df_review_cleaned")

2024-08-30 17:33:10,671 - INFO - Saving DataFrame to temporary directory...
2024-08-30 17:33:11,277 - INFO - DataFrame saved successfully to temporary directory.
2024-08-30 17:33:11,280 - INFO - File moved to final output path: /spark-data/Cleaned_Csv_File/df_review_cleaned.csv
2024-08-30 17:33:11,292 - INFO - Temporary directory removed: /spark-data/Cleaned_Csv_File/temp_dir


'/spark-data/Cleaned_Csv_File/df_review_cleaned.csv'

### Cleaning df_orders

In [68]:
df_order = dfs["df_data_orders"]

In [69]:
df_order.show()

+--------+-------------+-----+----------+--------+
|order_id|order_item_id|price|product_id|quantity|
+--------+-------------+-----+----------+--------+
|     868|            1|   70|      1571|       7|
|    3151|            2|    8|      3809|      -3|
|    1582|            3|   32|      4896|       8|
|    2578|            4|   64|      2712|       4|
|    2792|            5|   53|      1111|       4|
|    1257|            6|   69|      2874|       1|
|    2389|            7|   40|      2670|       9|
|    2013|            8|   39|      3779|       6|
|    4272|            9|   15|      3109|       5|
|    1260|           10|   21|      2110|       8|
|    3617|           11|   27|      3350|       1|
|    1479|           12|   96|      2917|       9|
|    1405|           13|   61|      1480|       5|
|    4526|           14|   69|      4686|       5|
|    2513|           15|    8|      3146|       8|
|    2346|           16|   92|      4182|       5|
|    3988|           17|   88| 

In [70]:
df_order_remove_duplicates = remove_duplicates(df_order)

2024-08-30 17:41:34,136 - INFO - Successfully removed duplicate rows based on all columns.


In [72]:
df_order_neg_handel = handle_negative_values(df_order_remove_duplicates, columns=["quantity"], operation="zero")

2024-08-30 17:51:03,754 - INFO - Successfully handled negative values in column 'quantity' by setting them to zero.
